In [5]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [6]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [7]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [8]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
1,model_3_5_0,0.952887,0.975711,0.858985,0.953538,0.052140,0.045376,0.069517,0.056737,"Hidden Size=[15, 7], regularizer=0.02, learnin..."
22,model_3_3_1,0.919504,0.691277,0.831082,0.801143,0.089085,0.164863,0.246864,0.203452,"Hidden Size=[15, 7], regularizer=0.02, learnin..."
23,model_3_2_6,0.919357,0.670385,0.745920,0.731572,0.089248,0.219666,0.241783,0.230071,"Hidden Size=[15, 7], regularizer=0.02, learnin..."
24,model_3_3_0,0.912418,0.931783,0.944081,0.943560,0.096928,0.036429,0.081723,0.057744,"Hidden Size=[15, 7], regularizer=0.02, learnin..."
31,model_3_3_2,0.905230,0.568047,0.804448,0.749187,0.104882,0.230670,0.285789,0.256608,"Hidden Size=[15, 7], regularizer=0.02, learnin..."
43,model_2_3_3,0.887388,0.461611,0.951066,0.856810,0.124628,0.245697,0.084079,0.169644,"Hidden Size=[15, 7], regularizer=0.1, learning..."
46,model_3_4_1,0.884468,0.977176,0.190367,0.936832,0.127860,0.040127,0.105959,0.071107,"Hidden Size=[15, 7], regularizer=0.02, learnin..."
65,model_3_4_0,0.881629,0.977160,0.193711,0.937001,0.131001,0.040154,0.105522,0.070916,"Hidden Size=[15, 7], regularizer=0.02, learnin..."
68,model_2_3_1,0.878767,0.858146,0.980969,0.958085,0.134169,0.064736,0.032699,0.049660,"Hidden Size=[15, 7], regularizer=0.1, learning..."
69,model_2_3_0,0.876680,0.871317,0.983815,0.962713,0.136479,0.058725,0.027808,0.044176,"Hidden Size=[15, 7], regularizer=0.1, learning..."
